In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [ ]:
//1. Local version: clone this repository, run `sbt publishLocal`
//2. Remote version: nothing to do (but you depend on what versions are available)
import $ivy.`com.github.dzufferey::scadla:0.1.1`

In [ ]:
import scadla._
import InlineOps._
import EverythingIsIn.{millimeters, degrees}
import scadla.backends.OpenSCAD // for rendering (getting a mesh)
import scadla.backends.almond.Viewer // to show the mesh in jupyter/almond

In [ ]:
// exmaple from the README
val c = Cube(1, 1, 1)
val s = Sphere(1.5)
val obj = (c + c.move(-0.5, -0.5, 0)) * s
val mesh = OpenSCAD(obj) // Solid -> Polyhedron
Viewer(mesh)

In [ ]:
// a more complex example
import squants.space.{Length, Angle}
import scala.language.postfixOps
import squants.space.LengthConversions._
import scadla.utils.{RoundedCubeH, Trig, Trapezoid}

// dimensions
val baseWidth = 95 mm
val baseLength = 150 mm
val baseDepth = 5 mm

val overallWidth = 200 mm
val overallLength = baseLength
val cornerRadius  = 10 mm
val wall = 2 mm
val wallStraight = 40 mm
val wallSlopped = 10 mm
val wallAngle: Angle = 45

val grooveWidth = 3 mm
val grooveDepth = 2 mm
val nbrGrooveH = 12
val nbrGrooveV = 10

def hat(x: Length, y: Length, z: Length, r: Length, a: Angle) = {
    val d = r*2
    val smaller = z * Trig.sin(a)
    Minkowski(
        Trapezoid((x-d-smaller,x-d,0),(y-d-smaller,y-d,0),z-1),
        Cylinder(r,1)
    ).move(r,r,0)
}

val base = Union(
    Cube(baseWidth, baseLength, baseDepth).move((overallWidth-baseWidth)/2,(overallLength-baseLength)/2,-baseDepth),
    RoundedCubeH(overallWidth, overallLength, wallStraight, cornerRadius),
    hat(overallWidth, overallLength, wallSlopped, cornerRadius, wallAngle).moveZ(wallStraight)
)
val sampleGrooveH = Cube(grooveWidth, overallLength - 2*wall - 10, grooveDepth)
val stepH = (overallWidth - 2*wall - 10) / nbrGrooveH
val groovesH = for (i <- 1 until nbrGrooveH) yield sampleGrooveH.move(wall + 5 + i * stepH - grooveWidth/2, wall+5, wall)
val sampleGrooveV = Cube(overallWidth - 2*wall - 10, grooveWidth, grooveDepth)
val stepV = (overallLength - 2*wall - 10) / nbrGrooveV
val groovesV = for (i <- 1 until nbrGrooveV) yield sampleGrooveV.move(wall + 5, wall + 5 + i * stepV - grooveWidth/2,wall)
val shell = Difference(
    base,
    RoundedCubeH(overallWidth - 2*wall, overallLength - 2*wall, wallStraight-wall-grooveDepth, cornerRadius-wall).move(wall,wall,wall+grooveDepth),
    hat(overallWidth - 2*wall, overallLength - 2*wall, wallSlopped, cornerRadius-wall, wallAngle).move(wall,wall,wallStraight),
    Union(groovesH: _*),
    Union(groovesV: _*)
)
val mesh = OpenSCAD(shell)
Viewer(mesh)

In [ ]:
// save the object as an STL file
scadla.backends.stl.Printer.storeBinary(mesh, "milling_basket.stl")